In [7]:
!pip install BeautifulSoup4
!pip install requests

In [23]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
sp = BeautifulSoup(page.text, 'lxml')
donnnes = []
colonnes = []
tbl = sp.find(class_='wikitable')
for index, tr in enumerate(tbl.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index == 0):
        colonnes = section
    else:
        donnnes.append(section)

#convert list into Pandas DataFrame
df = pd.DataFrame(data = donnnes,columns = colonnes)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [24]:
df = df[df['Borough']!= 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


C:\Users\LifeBook\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Postcode,Borough,Neighbourhood


In [25]:
df["Neighbourhood"] = df.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

df = df.drop_duplicates()

if(df.index.name != 'Postcode'):
    df = df.set_index('Postcode')
    
df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


In [26]:
df.shape

(103, 2)

In [29]:
url = 'https://cocl.us/Geospatial_data'
geo_df = pd.read_csv(url)


#rename columns and set the index to be Postcode
geo_df.columns = ["Postcode", "Latitude", "Longitude"]
if(geo_df.index.name != 'Postcode'):
    geo_df = geo_df.set_index('Postcode')
    
geo_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [30]:
df = df.join(geo_df)
df.head(11)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
M7A,Queen's Park,Not assigned,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M3B,North York,Don Mills North,43.745906,-79.352188
M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
